# 정확도
### Titanic 엉터리 분류

In [11]:
import numpy as np
from sklearn.base import BaseEstimator

In [14]:
# BaseEstimator 클래스의 자식 클래스

class MyDummyClassifier(BaseEstimator):
    # 따로 속성을 부여하지 않고 
    # fit(), method() 메소드만 Overrider할 것이기 때문에 생성자는 필요없음
    def fit(self, X, y):
        # 메소드 첫 인자 self여야 되는 걸 까먹다니...

        # "여성이면 무조건 생존이다"이기 때문에
        # 따로 모델 만들어 줄 필요 없이 메소드만 Override만 하면 됨
        pass
    def predict(self, X):
        # y^ 을 구하기 귀해 X(=X_test)가 인자로
        # pred= dt_clf(X)에서 test셋을 행x열 291x10 로 정했을 때
        # X.shape 하면 행 수만큼 나옴
        pred = np.zeros((X.shape[0],1))
        for i in range(X.shape[0]):
            # 행마다 돌아가면서 성별 확인. 0은 여성
            if X['Sex'].iloc[i] == 0:
                pred[i,0] = 1
            # 남자일 땐 이미 0(사망)으로 만들었기에 더이상 할 게 없음
        return pred     

In [15]:
X = np.zeros((10,1))
print(X.shape)
X

(10, 1)


array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [16]:
# 타이타닉 데이터
import pandas as pd
titanic_df = pd.read_csv('./data/train.csv')
y_titanic_df = titanic_df['Survived']
X_titanic_df = titanic_df.drop('Survived', axis= 1 )

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_titanic_df, y_titanic_df, test_size = 0.2, random_state=11
)

In [18]:
my_clf = MyDummyClassifier()
my_clf.fit(X_train,y_train)
my_pred = my_clf.predict(X_test)

In [19]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, my_pred)

0.659217877094972

### Null값이 섞여있으면 건너뛰고 하기 떄문에 정확도가 낮음

In [25]:
# 데이터 전처리에 필요한 함수
from sklearn.preprocessing import LabelEncoder

def fillna(df):
    df.Age.fillna(df.Age.mean(), inplace=True)
    df.Cabin.fillna(df.Cabin.describe()['top'], inplace=True)
    df.Embarked.fillna(df.Embarked.describe()['top'], inplace=True)
    df.Fare.fillna(df.Fare.mean(), inplace=True)
    return df

# 머신러닝 알고리즘에 불필요한 변수(Feature) 제거
def redundant_features(df):
    df.drop(['PassengerId','Name','Ticket'], axis=1, inplace=True)
    return df

# 레이블 인코딩
def label_features(df):
    features = ["Cabin","Sex","Embarked"]
    for feature in features:
        le = LabelEncoder()
        df[feature] = le.fit_transform(df[feature])
    return df

# 위의 함수를 총망라
def transform_features(df):
    df = fillna(df)
    df = redundant_features(df)
    df = label_features(df)
    return df


In [26]:
titanic_df = pd.read_csv('./data/train.csv')
y_titanic_df = titanic_df['Survived']
# 정답열인 Survived 열 제거
X_titanic_df = titanic_df.drop('Survived', axis= 1 )
# 전처리 함수
X_titanic_df = transform_features(X_titanic_df)

In [27]:
X_train, X_test, y_train, y_test = train_test_split(
    X_titanic_df, y_titanic_df, test_size = 0.2, random_state=11
)

In [28]:
my_clf = MyDummyClassifier()
my_clf.fit(X_train,y_train)
my_pred = my_clf.predict(X_test)

In [29]:
accuracy_score(y_test, my_pred)

0.8324022346368715

### MyFakeClassifier
- MNist 손글씨

In [31]:
class MyFakeClassifier(BaseEstimator):
    def fit(self, X, y):
        pass
    # 입력값으로 들어오는 X 데이터 셋의 크기만큼 모두 0값으로 반환
    def predict(self, X):
        return np.zeros( (len(X),1), dtype = bool)
    

In [38]:
from sklearn.datasets import load_digits

# 사이킷런의 내장 데이터셋 load_digits()을 이용하여 MNIST 데이터 로딩
digits = load_digits()

# digits 번호가 7번이면 True이고 이를 astype(int)로 1로 변환
# 7번이 아니면 False이고 0으로 변환
y = (digits.target == 7).astype(int)
X_train, X_test, y_train, y_test = train_test_split(
    digits.data, y, random_state=11
)

In [39]:
print(f'레이블 테스트 세트 크기: {y_test.shape}')
print(f'테스트 세트 레이블 0과 1의 분포도: {pd.Series(y_test).value_counts()}')

레이블 테스트 세트 크기: (450,)
테스트 세트 레이블 0과 1의 분포도: 0    405
1     45
dtype: int64


In [40]:
# Dummy Classfier로 학습/예측/정확도 평가
fakeclf = MyFakeClassifier()
fakeclf.fit(X_train, y_train)
fakepred = fakeclf.predict(X_test)
score = accuracy_score(y_test, fakepred)
print(f'모든 예측을 0으로 해도 정확도는: {score:3f}')

모든 예측을 0으로 해도 정확도는: 0.900000


### 오차 행렬(Confusion Matrix)<br>
1종오류: 틀린 걸 맞다고 하는 오류 (= 무고한 사람을 범죄자라 낙인하는 오류)<br>
2종오류: 맞는 걸 틀렸다고 하는 오류 (= 암 환자를 건강하다고 하는 오류) <br>
- MyFakeClassifier 사례

In [41]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, fakepred)
# TN,          FP(1종오류)
# FN(2종오류), TP

array([[405,   0],
       [ 45,   0]], dtype=int64)

- 정밀도(Precision) <br>
TP / (FP + TP) <br>
1종오류에 민감하게 반응. 즉 1종오류를 줄이기 위한 척도 <br>
판사가 잘한다 못한다는 정밀도에 따라

- 재현율(Precision) <br>
TP / (FN + TP) <br>
2종오류에 민감하게 반응. 즉 2종오류를 줄이기 위한 척도 <br>
의사가 잘한다 못한다는 정밀도에 따라 

- F1 Score <br>
정밀도와 재현율이 조화를 이루는 지점 (조화평균도 여기서 나옴)

In [49]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix

def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    
    print(f'오차 행렬: {confusion}')
    print(f'정확도: {accuracy:.4f}, 정밀도: {precision:.4f}, 재현율: {recall:.4f}')

In [50]:
titanic_df = pd.read_csv('./data/train.csv')
y_titanic_df = titanic_df['Survived']
# 정답열인 Survived 열 제거
X_titanic_df = titanic_df.drop('Survived', axis= 1 )
# 전처리 함수
X_titanic_df = transform_features(X_titanic_df)
X_train, X_test, y_train, y_test = train_test_split(
    X_titanic_df, y_titanic_df, test_size = 0.2, random_state=11
)

In [51]:
# DummyClassifier말고 Logistic Regression을 이용해 제대로 분류함
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression()
lr_clf.fit(X_train, y_train)
pred = lr_clf.predict(X_test)
get_clf_eval(y_test, pred)

오차 행렬: [[109   9]
 [ 14  47]]
정확도: 0.8715, 정밀도: 0.8393, 재현율: 0.7705


### Precision/Recall Trade-off

In [54]:
# 0, 1로 나눠진다고해서 그 값이 실제로 0, 1일까? 
# 아님. 언제까지나 0, 1,이 나올 확률이 크다는 얘기임
pred_proba = lr_clf.predict_proba(X_test)
""" 더 큰 값에따라서 0 아니면 1로 나온다
아래 pred[:10] 와 비교 """
pred_proba[:10]

array([[0.38437275, 0.61562725],
       [0.87923083, 0.12076917],
       [0.87566575, 0.12433425],
       [0.85366996, 0.14633004],
       [0.77514259, 0.22485741],
       [0.85188011, 0.14811989],
       [0.88716943, 0.11283057],
       [0.22730428, 0.77269572],
       [0.74699296, 0.25300704],
       [0.35305167, 0.64694833]])

In [55]:
pred[:10]

array([1, 0, 0, 0, 0, 0, 0, 1, 0, 1], dtype=int64)

In [57]:
from sklearn.preprocessing import Binarizer
x = [[1, -1, 2],
     [2, 0, 0],
     [0, 1.1, 1.2]]
# threshold 기준값보다 같거나 작으면 0을, 크면 1을 반환
binarizer = Binarizer(threshold = 1.1)
binarizer.fit_transform(x)

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.]])

- F1 스코어 = 조화평균 <br>
재현율과 정밀도가 조화를 이룰 때 높은 수치를 가진다

In [59]:
from sklearn.metrics import f1_score

f1 = f1_score(y_test, pred)
print(f'F1 스코어: {f1}')

F1 스코어: 0.8034188034188035
